In [1]:
import numpy as np
import pandas as pd

In [2]:
catalog = "../../Bases_de_datos/Data_catalog.csv"
iso = "../../Bases_de_datos/Country_ISO_code.csv"
path_v = "../data/variables.csv"
ix = "country_id"

df_c = pd.read_csv(catalog)
df_iso = pd.read_csv(iso)
data_s = pd.DataFrame(index = df_iso["alpha-3"])
data_s.index = data_s.index.set_names(ix)
data_c = data_s.copy()
data_g = data_s.copy()
data_q_s = data_s.copy()
data_q_c = data_s.copy()
data_q_g = data_s.copy()

df_v = pd.read_csv(path_v)

df_c[ df_c["Origin"] == "UNESCO UIS" ]


,Dataset,ID,Origin,Countries,Granularity,Timestep,Annex,Website,Filename,Path
20,Demographic and Socio-economic Indicators,DEM,UNESCO UIS,All,National,Yearly,NaN,https://uis.unesco.org/bdds,DEM_DATA_NATIONAL.csv,Bases_de_datos/UNESCO_UIS/DEM/
21,Other Policy Relevant Indicators (OPRI),OPRI,UNESCO UIS,All,National,Yearly,NaN,https://uis.unesco.org/bdds,OPRI_DATA_NATIONAL.csv,Bases_de_datos/UNESCO_UIS/OPRI/
22,Research and Development (R&D) – Other Policy ...,SCN-OPRI,UNESCO UIS,All,National,Yearly,NaN,https://uis.unesco.org/bdds,SCN-OPRI_DATA_NATIONAL.csv,Bases_de_datos/UNESCO_UIS/SCN-OPRI/
23,Research and Development (R&D) SDG 9.5,SCN-SDG,UNESCO UIS,All,National,Yearly,NaN,https://uis.unesco.org/bdds,SCN-SDG_DATA_NATIONAL.csv,Bases_de_datos/UNESCO_UIS/SCN-SDG/
24,SDG Global and Thematic Indicators,SDG,UNESCO UIS,All,National,Yearly,NaN,https://uis.unesco.org/bdds,SDG_DATA_NATIONAL.csv,Bases_de_datos/UNESCO_UIS/SDG/
25,SDG 11.4,SDG11,UNESCO UIS,All,National,Yearly,NaN,https://uis.unesco.org/bdds,SDG11_DATA_NATIONAL.csv,Bases_de_datos/UNESCO_UIS/SDG11/


In [13]:
for row in df_v[ df_v["Category"] == "socioeconomic" ].itertuples():
    v = row.Variable
    if row.Origin == "World Bank": nr = 3
    else: nr = 0
    
    df = pd.read_csv( "../../"
        + df_c.loc[df_c["ID"]==row.ID, 'Path'].iloc[0]
        + df_c.loc[df_c["ID"]==row.ID, "Filename" ].iloc[0],
        skiprows = nr, low_memory = False )
    
    if row.Origin == "OECD":
        df = df[ df["MEASURE"] == v ]
        df = df.rename(columns = {"REF_AREA": ix})
        df = df[ df[ix].isin(df_iso["alpha-3"]) ]
        
        if row.ID in ["DF_GREEN_GROWTH", "DF_NECRS", "DF_REVPOT"]:
            df_i = df[ (df["MEASURE"]==v) ]
            if v == "ECRATE":
                df_i = df_i[ (df["SECTOR"]==row.SECTOR)
                & (df["EMISSIONS_SOURCE"]==row.EMISSIONS_SOURCE) ]
            data_g[v] = df_i.sort_values( [ix, "TIME_PERIOD"]
                ).drop_duplicates(ix, keep="last").set_index(ix)["OBS_VALUE"]
            if v == "POP_NETMIGR": data_g[v] = np.abs( data_g[v] )
            
    elif row.Origin == "Global Data Lab":
        df = df.rename(columns = {"ISO_Code": ix})
        df = df[ df[ix].isin(df_iso["alpha-3"]) ]
        data_s[v] = df[ df["Level"] == "National" ].set_index(ix)[v]

    elif row.Origin == "World Bank":
        df = df.rename(columns = {"Country Code": ix})
        df = df[ df[ix].isin(df_iso["alpha-3"]) ]
        data_s[v] = df.set_index(ix).iloc[:, 4:-1].ffill(axis = 1).iloc[:, -1]

    elif row.Origin == "UNESCO UIS":
        if row.ID in ["DEM", "SCN-SDG"]:
            df.columns = [ x.lower() for x in df.columns ]
        df = df[ df[ix].isin(df_iso["alpha-3"]) ]
        if row.PERCENTAGE == "Yes": pop = data_s["200101"]
        else: pop = 1
        data_s[v] = ( df[ df["indicator_id"]==v ].sort_values( [ix, "year"]
            ).drop_duplicates(ix, keep="last").set_index(ix)["value"] / pop )
    
    # corregir random
    data_q_s[v] = ( 1 + pd.qcut( data_s[v]
        + np.random.random(data_s[v].shape[0])/10000, 20, labels = False ) )
    if row.Negative_indicator:
        data_q_s[v] = 21 - data_q_s[v]
        
data_q_s = data_q_s / 2
data_q_s["Mean"] = data_q_s.mean(axis = 1)
data_q_s["Count"] = data_q_s.count(axis = 1)
data_q_s = data_q_s[ data_q_s["Count"] > 1 ].sort_values("Mean")

data_q_s

,200101,200144,200151,DT.TDS.DECT.GN.ZS,NY.GDP.PCAP.CD,SH.DYN.AIDS.ZS,SI.POV.LMIC,SP.DYN.LE00.IN,SP.DYN.TFRT.IN,SP.POP.GROW,...,ROFST.H.1.F,ROFST.H.1.Q1,ROFST.H.1.RUR,XGDP.FSGOV,POP_NETMIGR,Subnational HDI,Subnational GDI,Gini index,Mean,Count
country_id,,,,,,,,,,,,,,,,,,,,,
VAT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,0.583333,3
SSD,4.0,10.0,9.0,NaN,1.5,1.5,0.5,10.0,1.0,3.0,...,0.5,0.5,0.5,0.5,2.5,0.5,0.5,2.0,2.865530,24
AGO,2.0,8.0,9.5,1.0,3.0,2.0,2.5,9.0,0.5,0.5,...,2.0,2.0,1.0,1.5,2.5,2.5,2.5,0.5,2.915064,26
TCD,3.0,8.5,10.0,8.5,1.0,2.5,1.5,10.0,0.5,0.5,...,0.5,0.5,0.5,2.0,5.0,0.5,0.5,4.5,3.031481,27
PNG,4.0,8.5,8.5,1.0,3.0,3.0,1.0,8.0,3.0,2.5,...,1.0,1.5,2.0,0.5,4.5,2.0,2.5,2.5,3.088148,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PLW,9.5,4.5,4.5,NaN,6.5,NaN,NaN,7.0,10.0,8.0,...,NaN,NaN,NaN,9.5,10.0,NaN,NaN,NaN,7.086310,14
QAT,6.5,0.5,10.0,NaN,10.0,7.0,NaN,2.0,6.5,8.0,...,6.5,10.0,10.0,3.0,2.5,8.0,10.0,NaN,7.092614,22
LUX,7.5,2.0,3.0,NaN,10.0,6.5,10.0,1.0,9.0,3.0,...,NaN,NaN,NaN,6.5,5.0,9.5,8.5,7.0,7.147870,21


In [ ]:
for row in df_v[ (df_v["Category"] == "climate") ].itertuples():
    print(row)
    v = row.Variable
    if row.Origin == "World Bank": nr = 3
    else: nr = 0
    
    df = pd.read_csv( "../../"
        + df_c.loc[df_c["ID"]==row.ID, 'Path'].iloc[0]
        + df_c.loc[df_c["ID"]==row.ID, "Filename" ].iloc[0],
        skiprows = nr, low_memory = False )
    
    if row.Origin == "OECD":
        df = df[ df["MEASURE"] == v ]
        df = df.rename(columns = {"REF_AREA": ix})
        df = df[ df[ix].isin(df_iso["alpha-3"]) ]
        
        df_i = df[ (df["MEASURE"]==v) ]
        if v == "POP_NETMIGR": df_i["OBS_VALUE"] = np.abs( df_i["OBS_VALUE"] )
        if v == "ECRATE":
            df_i = df_i[ (df["SECTOR"]==row.SECTOR)
            & (df["EMISSIONS_SOURCE"]==row.EMISSIONS_SOURCE) ]
        elif v == "HD_POP_EXP":
            df_i["OBS_VALUE"] = df_i.groupby("DURATION").sum()["OBS_VALUE"]
        data_g[v] = df_i.sort_values( [ix, "TIME_PERIOD"]
            ).drop_duplicates(ix, keep="last").set_index(ix)["OBS_VALUE"]

    elif row.Origin == "Global Data Lab":
        df = df.rename(columns = {"ISO_Code": ix})
        df = df[ df[ix].isin(df_iso["alpha-3"]) ]
        data_g[v] = df[ df["Level"] == "National" ].set_index(ix)[v]
    
    elif row.Origin == "World Bank":
        df = df.rename(columns = {"Country Code": ix})
        df = df[ df[ix].isin(df_iso["alpha-3"]) ]
        data_g[v] = df.set_index(ix).iloc[:, 4:-1].ffill(axis = 1).iloc[:, -1]
    
    elif row.Origin == "UNESCO UIS":
        if row.ID in ["DEM", "SCN-SDG", "SDG11"]:
            df.columns = [ x.lower() for x in df.columns ]
        df = df[ df[ix].isin(df_iso["alpha-3"]) ]
        if row.PERCENTAGE == "Yes": pop = data_g["200101"]
        else: pop = 1
        data_g[v] = ( df[ df["indicator_id"]==v ].sort_values( [ix, "year"]
            ).drop_duplicates(ix, keep="last").set_index(ix)["value"] / pop )
    
    # corregir random
    data_q_g[v] = ( 1 + pd.qcut( data_g[v]
        + np.random.random(data_g[v].shape[0])/10000, 20, labels = False ) )
    if row.Negative_indicator:
        data_q_g[v] = 21 - data_q_g[v]

data_q_g = data_q_g / 2
data_q_g["Mean"] = data_q_g.mean(axis = 1)
data_q_g["Count"] = data_q_g.count(axis = 1)
data_q_g = data_q_g[ data_q_g["Count"] > 1 ].sort_values("Mean")

data_q_g

In [9]:
for row in df_v[ (df_v["Category"] == "governance") ].itertuples():
    print(row)
    v = row.Variable
    if row.Origin == "World Bank": nr = 3
    else: nr = 0
    
    df = pd.read_csv( "../../"
        + df_c.loc[df_c["ID"]==row.ID, 'Path'].iloc[0]
        + df_c.loc[df_c["ID"]==row.ID, "Filename" ].iloc[0],
        skiprows = nr, low_memory = False )
    
    if row.Origin == "OECD":
        df = df[ df["MEASURE"] == v ]
        df = df.rename(columns = {"REF_AREA": ix})
        df = df[ df[ix].isin(df_iso["alpha-3"]) ]
        
        df_i = df[ (df["MEASURE"]==v) ]
        if v == "ECRATE":
            df_i = df_i[ (df["SECTOR"]==row.SECTOR)
            & (df["EMISSIONS_SOURCE"]==row.EMISSIONS_SOURCE) ]
        data_g[v] = df_i.sort_values( [ix, "TIME_PERIOD"]
            ).drop_duplicates(ix, keep="last").set_index(ix)["OBS_VALUE"]
        if v == "POP_NETMIGR": data_g[v] = np.abs( data_g[v] )
    
    elif row.Origin == "Global Data Lab":
        df = df.rename(columns = {"ISO_Code": ix})
        df = df[ df[ix].isin(df_iso["alpha-3"]) ]
        data_g[v] = df[ df["Level"] == "National" ].set_index(ix)[v]
    
    elif row.Origin == "World Bank":
        df = df.rename(columns = {"Country Code": ix})
        df = df[ df[ix].isin(df_iso["alpha-3"]) ]
        data_g[v] = df.set_index(ix).iloc[:, 4:-1].ffill(axis = 1).iloc[:, -1]
    
    elif row.Origin == "UNESCO UIS":
        if row.ID in ["DEM", "SCN-SDG", "SDG11"]:
            df.columns = [ x.lower() for x in df.columns ]
        df = df[ df[ix].isin(df_iso["alpha-3"]) ]
        if row.PERCENTAGE == "Yes": pop = data_g["200101"]
        else: pop = 1
        data_g[v] = ( df[ df["indicator_id"]==v ].sort_values( [ix, "year"]
            ).drop_duplicates(ix, keep="last").set_index(ix)["value"] / pop )
    
    # corregir random
    data_q_g[v] = ( 1 + pd.qcut( data_g[v]
        + np.random.random(data_g[v].shape[0])/10000, 20, labels = False ) )
    if row.Negative_indicator:
        data_q_g[v] = 21 - data_q_g[v]

data_q_g = data_q_g / 2
data_q_g["Mean"] = data_q_g.mean(axis = 1)
data_q_g["Count"] = data_q_g.count(axis = 1)
data_q_g = data_q_g[ data_q_g["Count"] > 1 ].sort_values("Mean")

data_q_g

Pandas(Index=50, Number=51, Origin='OECD', Category='governance', ID='DF_NECRS', Dataset_name='Net effective carbon rates', Negative_indicator=False, Variable='ECRATE', Name='Effective carbon rate', Note=nan, Unit_of_measure='Euros per tonne of CO2-equivalent', HEAT_STRESS=nan, Heat_stress_thresholds=nan, RET_PERIOD=nan, Return_periods=nan, UNIT_MEASURE=nan, EXPOSURE_LEVEL=nan, Exposure_levels=nan, DURATION=nan, Duration=nan, SECTOR='_T', Economic_sector='Total', EMISSIONS_SOURCE='_T', Emissions_source='All sources', _24=nan, PERCENTAGE=nan)
Pandas(Index=51, Number=52, Origin='OECD', Category='governance', ID='DF_REVPOT', Dataset_name='Net energy tax revenues and reform potential', Negative_indicator=True, Variable='FSUBS', Name='Fossil fuel subsidies', Note=nan, Unit_of_measure='Percentage of GDP', HEAT_STRESS=nan, Heat_stress_thresholds=nan, RET_PERIOD=nan, Return_periods=nan, UNIT_MEASURE=nan, EXPOSURE_LEVEL=nan, Exposure_levels=nan, DURATION=nan, Duration=nan, SECTOR=nan, Economic_

,CO2_AIRTRACAP,LCC_CROPTOARTIF,LCC_NATTOCROP,SW_PERMTONOT,CO2_DBEMCAP,GPAT_DE,NRG_INT,LCC_NATGAIN,PA_MARINE,MWAS_INT,...,WLIFE_MA,"Patent applications, residents",Scientific and technical journal articles,HEXPCSTPPPCAP.CULHER,HEXPCSTPPPCAP.NATHER,EXPGDP.TOT,Mean,Count,ECRATE,FSUBS
country_id,,,,,,,,,,,,,,,,,,,,,
GRD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,3.0,NaN,NaN,NaN,1.833333,3,NaN,NaN
SXM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.937500,4,NaN,NaN
GUM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.5,2.083333,3,NaN,NaN
LIE,NaN,0.5,4.0,NaN,NaN,3.0,NaN,1.0,NaN,NaN,...,NaN,NaN,2.5,NaN,NaN,NaN,2.164286,7,NaN,NaN
MAC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.5,NaN,NaN,NaN,6.0,2.718750,4,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZWE,9.0,9.0,3.0,7.5,NaN,10.0,7.5,5.0,NaN,NaN,...,NaN,3.0,5.5,NaN,NaN,NaN,6.534965,13,NaN,NaN
GHA,9.0,3.5,2.5,8.0,NaN,9.5,9.5,7.5,NaN,NaN,...,NaN,3.5,7.0,NaN,NaN,5.0,6.582031,16,8.0,8.0
COD,10.0,7.5,5.0,6.0,NaN,NaN,10.0,5.5,NaN,NaN,...,NaN,5.0,4.0,NaN,NaN,5.0,6.734266,13,NaN,NaN


In [ ]:
# OECD
# EXT_TEMP: Exposure to extreme temperature

n = 6
file = f"../{df_c.loc[n, "Path"]}{df_c.loc[n, "Filename"]}"
df = pd.read_csv(file)
df = df.rename( columns = {"REF_AREA": ix} )
df = df[ df[ix].isin(df_iso["alpha-3"]) ]

vars = [
    "TN_POP_EXP",
    "HD_TN_POP_EXP",
    "HD_POP_EXP",
    "ID_POP_EXP",
    "UTCI_POP_EXP",
    "TEMP_CHANGE"
    ]
names = [
    "Population exposure to tropical nights",
    "Population exposure to days identified to be a hot summer day and tropical night",
    "Population exposure to hot days",
    "Population exposure to icing days",
    "Mean population exposure to heat stress",
    "Annual temperature change",
    ]

for v in vars:
    data_c[v] = df[ df["MEASURE"]==v ].sort_values( [ix, "TIME_PERIOD"]
        ).drop_duplicates(ix, keep="last").set_index(ix)["OBS_VALUE"]
    data_q_c[v] = pd.qcut( data_c[v]
            + np.random.random(data_c[v].shape[0])/10000, 20, labels = False )

FileNotFoundError: [Errno 2] No such file or directory: '../Bases_de_datos/OECD_Air_and_Climate/OECD.ENV.EPI,DSD_ECH@EXT_TEMP,1.1+all.csv'

In [ ]:
n = 6
file = f"../{df_c.loc[n, "Path"]}{df_c.loc[ n, "Filename" ]}"
df = pd.read_csv(file)
df[ df["REF_AREA"].isin(df_iso["alpha-3"]) ]

,STRUCTURE,STRUCTURE_ID,STRUCTURE_NAME,ACTION,REF_AREA,Reference area,FREQ,Frequency of observation,MEASURE,Measure,...,TIME_PERIOD,Time period,OBS_VALUE,Observation value,OBS_STATUS,Observation status,UNIT_MULT,Unit multiplier,DECIMALS,Decimals
386,DATAFLOW,OECD.ENV.EPI:DSD_ECH@EXT_TEMP(1.1),Exposure to extreme temperature,I,SXM,Sint Maarten,A,Annual,ID_POP_EXP,Population exposure to icing days,...,1979,NaN,0.0,NaN,NaN,NaN,0,Units,2,Two
387,DATAFLOW,OECD.ENV.EPI:DSD_ECH@EXT_TEMP(1.1),Exposure to extreme temperature,I,SXM,Sint Maarten,A,Annual,ID_POP_EXP,Population exposure to icing days,...,1980,NaN,0.0,NaN,NaN,NaN,0,Units,2,Two
388,DATAFLOW,OECD.ENV.EPI:DSD_ECH@EXT_TEMP(1.1),Exposure to extreme temperature,I,SXM,Sint Maarten,A,Annual,ID_POP_EXP,Population exposure to icing days,...,1981,NaN,0.0,NaN,NaN,NaN,0,Units,2,Two
389,DATAFLOW,OECD.ENV.EPI:DSD_ECH@EXT_TEMP(1.1),Exposure to extreme temperature,I,SXM,Sint Maarten,A,Annual,ID_POP_EXP,Population exposure to icing days,...,1982,NaN,0.0,NaN,NaN,NaN,0,Units,2,Two
390,DATAFLOW,OECD.ENV.EPI:DSD_ECH@EXT_TEMP(1.1),Exposure to extreme temperature,I,SXM,Sint Maarten,A,Annual,ID_POP_EXP,Population exposure to icing days,...,1983,NaN,0.0,NaN,NaN,NaN,0,Units,2,Two
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1125645,DATAFLOW,OECD.ENV.EPI:DSD_ECH@EXT_TEMP(1.1),Exposure to extreme temperature,I,FLK,Falkland Islands (Malvinas),A,Annual,TN_POP_EXP,Population exposure to tropical nights,...,2018,NaN,0.0,NaN,NaN,NaN,0,Units,2,Two
1125646,DATAFLOW,OECD.ENV.EPI:DSD_ECH@EXT_TEMP(1.1),Exposure to extreme temperature,I,FLK,Falkland Islands (Malvinas),A,Annual,TN_POP_EXP,Population exposure to tropical nights,...,2019,NaN,0.0,NaN,NaN,NaN,0,Units,2,Two
1125647,DATAFLOW,OECD.ENV.EPI:DSD_ECH@EXT_TEMP(1.1),Exposure to extreme temperature,I,FLK,Falkland Islands (Malvinas),A,Annual,TN_POP_EXP,Population exposure to tropical nights,...,2020,NaN,0.0,NaN,NaN,NaN,0,Units,2,Two
1125648,DATAFLOW,OECD.ENV.EPI:DSD_ECH@EXT_TEMP(1.1),Exposure to extreme temperature,I,FLK,Falkland Islands (Malvinas),A,Annual,TN_POP_EXP,Population exposure to tropical nights,...,2021,NaN,0.0,NaN,NaN,NaN,0,Units,2,Two
